<div align="center">
    <div><img src="../assets/redis_logo.svg" style="width: 130px"> </div>
    <div style="display: inline-block; text-align: center; margin-bottom: 10px;">
        <span style="font-size: 36px;"><b>Introduction to Redis Python</b></span>
        <br />
    </div>
    <br />
</div>

This notebook introduces [Redis](https://redis.io) and the standard Python client, [redis-py](https://redis-py.readthedocs.io/en/stable/), for interacting with the database. We will explore the basics of Redis setup, data structures, and capabilities like vector search!


### Install Python Dependencies

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
dir_path = os.getcwd()
parent_directory = os.path.dirname(dir_path)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["ROOT_DIR"] = parent_directory
print(dir_path)
print(parent_directory)

/Users/michael.yuan/work/genai/financial-rag-workshop/1_getting_started
/Users/michael.yuan/work/genai/financial-rag-workshop


In [2]:
%pip install -r $ROOT_DIR/requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
dir_path = os.getcwd()
parent_directory = os.path.dirname(dir_path)
# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## Hello World Redis

Now let's connect to the Redis db and get a basic feel for the most common
commands and data structures.

In [4]:
import redis
import json
import numpy as np

from time import sleep

# Connect with the Redis Python Client
client = redis.Redis.from_url(REDIS_URL)

client.ping()

True

In [5]:
client.dbsize() # should be empty

0

Redis, at it's core, is a simple key/value store. It supports a number of interesting
and flexible data structures that can solve a variatey of business and operational
problems.

### Strings

The basic string data type can be accessed using set/get methods. You can also place a
TTL policy (expiration) on any key in Redis.

In [6]:
client.set("hello", "world")

True

In [7]:
client.get("hello")

b'world'

In [8]:
client.delete("hello")

1

In [9]:
client.set("hello", "world")
client.expire("hello", time=3)

sleep(4)

# should be EMPTY
client.get("hello")

### Hashes
Hashes are collections of key/value pairs that are grouped together. It gets
serialized as a string in Redis, but can hold a variety of data in each field.

You can think of a Hash as a one-level deep Python dictionary.


In [10]:
obj = {
    "user": "john",
    "age": 45,
    "job": "dentist",
    "bio": "long form text of john's bio",
    "user_embedding": np.array([0.3, 0.4, -0.8], dtype=np.float32).tobytes() # cast vectors to bytes string
}

In [11]:
client.hset("user:john", mapping=obj)

5

In [12]:
client.hgetall("user:john")

{b'user': b'john',
 b'age': b'45',
 b'job': b'dentist',
 b'bio': b"long form text of john's bio",
 b'user_embedding': b'\x9a\x99\x99>\xcd\xcc\xcc>\xcd\xccL\xbf'}

In [13]:
client.delete("user:john")

1

### JSON
With the JSON capabilitie enabled, Redis can be a drop-in replacement for MongoDB
or other slower document databases. You can store nested and structured JSON data
directly in Redis.

In [14]:
# set a JSON obj
obj = {
    "user": "john",
    "metadata": {
        "age": 45,
        "job": "dentist",
    },
    "user_embedding": [0.3, 0.4, -0.8]
}

client.json().set("user:john", "$", obj)

True

In [15]:
# get user JSON obj
client.json().get("user:john")

{'user': 'john',
 'metadata': {'age': 45, 'job': 'dentist'},
 'user_embedding': [0.3, 0.4, -0.8]}

In [16]:
# grab array length for embedding field
client.json().arrlen("user:john", "$.user_embedding")

[3]

In [17]:
# grab obj keys
client.json().objkeys("user:john", "$")

[[b'user', b'metadata', b'user_embedding']]

In [18]:
# delete user JSON
client.delete("user:john")

1

### Lists
Lists store sequences of information... potentially list of messages in an LLM
converstion flow, or really any list of items in a queue.

In [19]:
# add items to a list
client.rpush("messages:john", *[
    json.dumps({"role": "user", "content": "Hello what can you do for me?"}),
    json.dumps({"role": "assistant", "content": "Hi, I am a helpful virtual assistant."})
])

2

In [20]:
# list all items in the list using indices
[json.loads(msg) for msg in client.lrange("messages:john", 0, -1)]

[{'role': 'user', 'content': 'Hello what can you do for me?'},
 {'role': 'assistant', 'content': 'Hi, I am a helpful virtual assistant.'}]

In [21]:
# count items in the list
client.llen("messages:john")

2

In [22]:
# pop the first item from the list and push to another list
client.rpoplpush("messages:john", "read_messages:john")

b'{"role": "assistant", "content": "Hi, I am a helpful virtual assistant."}'

In [23]:
client.lrange("read_messages:john", 0, -1)

[b'{"role": "assistant", "content": "Hi, I am a helpful virtual assistant."}']

In [24]:
# list cleanup
client.delete("messages:john", "read_messages:john")

2

### Pipelines
All Redis commands can be pipelined to gain some round trip latency improvements.

In [25]:
with client.pipeline(transaction=False) as pipe:
    for i in range(50):
        pipe.json().set(f"user:{i}", "$", obj)
    # execute batch
    pipe.execute()

In [26]:
client.dbsize()

50

In [27]:
# clean up!
client.flushall()

True

## Intro to Vector Search in Redis

Now that we have the basics down, we will dive into the fundamentals of vector
search in Redis using the base Python client.

### Dataset Preparation (PDF Documents)

To best demonstrate Redis as a vector database layer, we will load a single
financial (10k filings) doc and preprocess it using some helpers from LangChain:

- `UnstructuredFileLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter


In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

# Load list of pdfs from a folder
data_path = f"{parent_directory}/resources/10K"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

Listing available documents ... ['/Users/michael.yuan/work/genai/financial-rag-workshop/resources/10K/nke-10k-2023.pdf']


In [29]:
# fetch the Nike PDF
doc = [doc for doc in docs if "nke" in doc][0]

# set up the file loader/extractor and text splitter to create chunks
loader = UnstructuredFileLoader(doc, mode="single", strategy="fast")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=0)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

Done preprocessing. Created 180 chunks of the original pdf /Users/michael.yuan/work/genai/financial-rag-workshop/resources/10K/nke-10k-2023.pdf


In [30]:
# Take a look at content from a chunk
print(chunks[25].page_content)

NIKE is a consumer products company and the relative popularity of various sports and fitness activities and changing design trends affect the demand for our products, services and experiences. The athletic footwear, apparel and equipment industry is highly competitive both in the United States and worldwide. We compete internationally with a significant number of athletic and leisure footwear companies, athletic and leisure apparel companies, sports equipment companies, private labels and large companies that have diversified lines of athletic and leisure footwear, apparel and equipment. We also compete with other companies for the production capacity of contract manufacturers that produce our products. In addition, we and our contract manufacturers compete with other companies and industries for raw materials used in our products. Our NIKE Direct operations, both through our digital commerce operations and retail stores, also compete with multi-brand retailers, which sell our product

### Text embedding generation with SentenceTransformers

#### SentenceTransformer Models Cache folder
We are using `SentenceTransformer` in this demo and here we specify the cache folder. If you already downloaded the models in a local file system, set this folder here, otherwise the library tries to download the models in this folder if not available locally.

In particular, these models will be downloaded if not present in the cache folder:

models/models--sentence-transformers--all-MiniLM-L6-v2


In [31]:
#setting the local downloaded sentence transformer models f
os.environ["TRANSFORMERS_CACHE"] = f"{parent_directory}/models"

In [32]:
from sentence_transformers import SentenceTransformer

# load model - may take a minute or two to download the first time
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', cache_folder=os.getenv("TRANSFORMERS_CACHE", f"{parent_directory}/models"))

.gitattributes: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [33]:
%%time

# create embeddings
chunk_embeddings = model.encode([chunk.page_content for chunk in chunks])
len(chunk_embeddings) == len(chunks)

CPU times: user 12.1 s, sys: 2.49 s, total: 14.6 s
Wall time: 3.61 s


True

In [34]:
print(f"embedding dim should be {len(chunk_embeddings[0])}")

embedding dim should be 384


### Set up some helper functions

Helper functions to encode the single query vector and display redis search results

In [35]:
import pandas as pd


def encode_one(input):
    return model.encode(input).astype(np.float32).tobytes()


def table_view(res):
    if res.total == 0:
        print("No documents found.")
        return None
    
    res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
    return res_df


### Define a schema and create an index
Below we connect to Redis and create an index for vector search that contains a single text field and vector field.

In [36]:
from redis.commands.search.field import TagField, TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


index_name = "redispy"
key_prefix = f"doc:{index_name}"


def create_index(index_type: str = "FLAT"):       # Creates a FLAT index by default
    try:
        # check to see if index exists
        client.ft(index_name).info()
        print("Index already exists!")
    except:
        # define schema
        schema = (
            TagField("doc_id"),                    # Tag Field - synthetic ID
            TextField("content"),                  # Text Field
            VectorField("chunk_vector",            # Vector Field
                index_type, {                      # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",
                    "DIM": 384,                    # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.HASH)

        # create Index
        client.ft(index_name).create_index(fields=schema, definition=definition)

In [37]:
# Create the index
create_index()

In [38]:
# Check the info related to the newly created index
client.ft(index_name).info()

{'index_name': 'redispy',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b'doc:redispy'],
  b'default_score',
  b'1',
  b'indexes_all',
  b'false'],
 'attributes': [[b'identifier',
   b'doc_id',
   b'attribute',
   b'doc_id',
   b'type',
   b'TAG',
   b'SEPARATOR',
   b','],
  [b'identifier',
   b'content',
   b'attribute',
   b'content',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'chunk_vector',
   b'attribute',
   b'chunk_vector',
   b'type',
   b'VECTOR',
   b'algorithm',
   b'FLAT',
   b'data_type',
   b'FLOAT32',
   b'dim',
   384,
   b'distance_metric',
   b'COSINE']],
 'num_docs': 0,
 'max_doc_id': 0,
 'num_terms': 0,
 'num_records': 0,
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0',
 'total_inverted_index_blocks': 0,
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0.01532745361328125',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '2.288818359375e-5',
 'tag_overhead_sz_mb': '0',
 'text_overhead_

### Process and load data using Redis
Below we use a Redis pipeline (not a transaction) to batch send writes to Redis. This method helps with throughput significantly. The batch_size param can be customized and benchmarked on your hardware and with your data. We typically recommend starting small (100-200) and increasing as needed.

In [39]:
# load expects an iterable of dictionaries

batch_size = 200

with client.pipeline(transaction=False) as pipe:
    for i, chunk in enumerate(chunks):
        data = {
            'doc_id': f"{i}",
            'content': chunk.page_content,
            # For HASH -- must convert embeddings to bytes
            'chunk_vector': np.array(chunk_embeddings[i]).astype(np.float32).tobytes()
        }
        pipe.hset(f"{key_prefix}:{i}", mapping=data)
        # execute in "mini batches"
        if i % batch_size == 0:
            res = pipe.execute()

    # cleanup final batch execution
    res = pipe.execute()

In [40]:
# check the data size in Redis
len(chunks) == client.dbsize()

True

In [41]:
client.hgetall(f"{key_prefix}:0")

{b'chunk_vector': b'\xb5\x1fh\xbd\xa6\xe2\x00\xbe.\xba\xa5\xbc\x1d\\\t=\xe0\xa9\x8f<#\x95\xcf= \x14\xb9<\xe8\x9d\xb0<\xf6\xf94;\xb2O\x1a\xbd,h\x07=\x17\xe3\xb4=\xc1\xe8\x9a\xbd{\ny\xbdN\r;;\x93~G=\xa8\xdb\xa3\xbb=\xf9\xc5\xbc`\xdcA\xbd\x06\x9aH=m\x8b\x1b=\xa0O\xb8\xbd\x9fg\x95\xbc\xfb\t\xe7:an5<2$q\xbc\x13\xdb\x98\xbc\xb5~h=\xc1\xb62\xbd\x92\xf4\xfa\xbcW\x89{\xbd\x1b\xec\xc2\xbc\xae\xd5\xc3=s\x12x=\x875\xfb=\x8b\x08B\xbd\xa6x\xc7=\xf9I\x91\xbc\xf2\x952=tT5=\x88t\x97<\x14I\x92\xbd\xd2\xed\x95\xbd\xec\x87\xa4=F\xba\xbf\xbc<\x1b*=\x9f\x05^\xbd\x9e\x15B=v>6\xbc\xbf\xf4\xaf=1!\x97;4\xcb4<\x81\xfd\xf4<b\x11\xae<p?,\xbci\xd1\xb6\xbcY\x9bt\xbcu\x1eQ=X\x88\xc2<\x81\x03\xdc<\xe9^\xfd<\xc71\x8a\xbd.\x04\'\xbd\xe1\xe64=\x17\x1a\n=\'\xdd\xba\xba\x07i\xb49+\x05\r\xbd\xfd\xfd\xf8<I\xa1\xf0\xbd)\xe5\xb2\xbc\xc8\x8c_\xba|I\x83\xbd;\xc0W=\xbd\r\x96\xbd\xbdZ\xf09\xe9\xf4\xad\xbc\xa2\x8d\xc4=\x9a\xb6;<ST\x97\xbd\xd0;\x8c\xbd\xff\x02\xbd<\xf8\xf1h:\x05\xde\xc4;\xa8\x03\x1b=\xf7\xd7\x8b\xbc\xffe\xb8\xbb\x84

### Query the database
Now we can use the Redis search index to perform vector similarity search operations.

The code below takes a user input, converts to embeddings, and fetches the top 2 most semantically similar chunks from Redis.

In [42]:
# Grab user input
_input = "Nike profit margins and company performance"

query = (
    Query("*=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": np.array(model.encode(_input), dtype=np.float32).tobytes()
}

res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,content
0,doc:redispy:85,0.321347296238,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."
1,doc:redispy:84,0.328033447266,As discussed in Note 15 — Operating Segments a...


In [43]:
# Example of sorting by a field other than vector_distance
query = (
    Query("*=>[KNN 4 @chunk_vector $vector as vector_distance]")
     .sort_by("doc_id")
     .return_fields("doc_id", "content", "vector_distance")
     .paging(0, 4)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,doc_id,content
0,doc:redispy:118,0.3587500453,118,"NIKE, INC. CONSOLIDATED STATEMENTS OF INCOME\n..."
1,doc:redispy:158,0.360825419426,158,Tax (expense) benefit Gain (loss) net of tax\n...
2,doc:redispy:84,0.328033447266,84,As discussed in Note 15 — Operating Segments a...
3,doc:redispy:85,0.321347296238,85,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."


### Range Queries
Range queries allow you to set a pre defined "threshold" for which we want to return documents

In [44]:
query = (
    Query("@chunk_vector:[VECTOR_RANGE $radius $vector]=>{$YIELD_DISTANCE_AS: vector_distance}")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)
table_view(res)

,id,vector_distance,content
0,doc:redispy:85,0.321347296238,"TOTAL NIKE BRAND Converse\n\n$\n\n1,932 (4,841..."
1,doc:redispy:84,0.328033447266,As discussed in Note 15 — Operating Segments a...
2,doc:redispy:118,0.3587500453,"NIKE, INC. CONSOLIDATED STATEMENTS OF INCOME\n..."
3,doc:redispy:158,0.360825419426,Tax (expense) benefit Gain (loss) net of tax\n...
4,doc:redispy:81,0.363789558411,"NIKE Brand revenues, which represented over 90..."
5,doc:redispy:82,0.370122730732,"Lower margin in our NIKE Direct business, driv..."
6,doc:redispy:80,0.386211395264,"4,780 (508)\n\n7 % -80 %\n\nTOTAL NIKE BRAND W..."
7,doc:redispy:86,0.393619000912,Apparel revenues increased 9% on a currency-ne...
8,doc:redispy:89,0.394768118858,"3 % -4 %\n\n13 % 4 %\n\n1,494 190\n\n8 % 23 %\..."
9,doc:redispy:159,0.395310044289,ASIA PACIFIC & LATIN AMERICA\n\n(1)\n\nGLOBAL ...


### Add filter statements
Redis queries can contain both vector search and traditional filters (numeric, tags, text, geo) in one single command.

In [45]:
# filter for docs that contain "profit" in the content field and do KNN vector search
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)
}

res = client.ft(index_name).search(query, query_params)
table_view(res)

,id,vector_distance,content
0,doc:redispy:81,0.363789558411,"NIKE Brand revenues, which represented over 90..."
1,doc:redispy:74,0.453679800034,"NIKE designs, develops, markets and sells athl..."


In [46]:
# lets clean up our index
client.ft(index_name).dropindex(True)

b'OK'

### What about JSON Support?

Redis also allows you to store data in JSON objects. The JSON fields can contain metadata and vectors. Below is a simple example of indexing JSON data.

In [47]:
# schema
schema = (
    TextField("$.content",                     # Text Field (JSON path)
        as_name="content"                      # Text Field Alias -- required for JSON
    ),
    VectorField("$.chunk_vector",              # Vector Field (JSON path)
        "FLAT", {                              # Vector Index Type: FLAT or HNSW
            "TYPE": "FLOAT32",
            "DIM": 384,                        # Number of Vector Dimensions
            "DISTANCE_METRIC": "COSINE",       # Vector Search Distance Metric
        },
        as_name="chunk_vector"                 # Vector Field Alias -- required for JSON
    ),
)

# index Definition
definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.JSON) # select JSON here

# create Index
client.ft(index_name).create_index(fields=schema, definition=definition)

b'OK'

In [48]:
client.ft(index_name).info()

{'index_name': 'redispy',
 'index_options': [],
 'index_definition': [b'key_type',
  b'JSON',
  b'prefixes',
  [b'doc:redispy'],
  b'default_score',
  b'1',
  b'indexes_all',
  b'false'],
 'attributes': [[b'identifier',
   b'$.content',
   b'attribute',
   b'content',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'$.chunk_vector',
   b'attribute',
   b'chunk_vector',
   b'type',
   b'VECTOR',
   b'algorithm',
   b'FLAT',
   b'data_type',
   b'FLOAT32',
   b'dim',
   384,
   b'distance_metric',
   b'COSINE']],
 'num_docs': 0,
 'max_doc_id': 0,
 'num_terms': 0,
 'num_records': 0,
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0',
 'total_inverted_index_blocks': 0,
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0.01532745361328125',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '2.288818359375e-5',
 'tag_overhead_sz_mb': '0',
 'text_overhead_sz_mb': '0',
 'total_index_memory_sz_mb': '0.015350341796875',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg

In [49]:
# Write JSON data to the index

batch_size = 200

with client.pipeline(transaction=False) as pipe:
    for i, chunk in enumerate(chunks):
        redis_key = f"{key_prefix}:{i}"
        data = {
            'content': chunk.page_content,
            'chunk_vector': chunk_embeddings[i].tolist() # notice that we don't need to convert JSON embeddings to bytes
        }
        #print(data)
        pipe.json().set(redis_key, "$", data)
        # mini batch
        if i % batch_size == 0:
            res = pipe.execute()

    res = pipe.execute() # make sure to use mini batches if working with larger datasets

In [50]:
# Fetch the JSON doc
client.json().get(f"{key_prefix}:0", "$")

[{'content': "NIKE, Inc.(Exact name of Registrant as specified in its charter)Oregon93-0584541(State or other jurisdiction of incorporation)(IRS Employer Identification No.)One Bowerman Drive, Beaverton, Oregon 97005-6453(Address of principal executive offices and zip code)(503) 671-6453(Registrant's telephone number, including area code)SECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:Class B Common StockNKENew York Stock Exchange(Title of each class)(Trading symbol)(Name of each exchange on which registered)SECURITIES REGISTERED PURSUANT TO SECTION 12(G) OF THE ACT:NONE\n\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-K(Mark One)☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934FOR THE FISCAL YEAR ENDED MAY 31, 2023OR☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934FOR THE TRANSITION PERIOD FROM TO .Commission File No. 1-10635\n\nAs of November 30, 20

In [51]:
# And now you can perform the same kinds of queries
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vector as vector_distance]")
     .sort_by("vector_distance")
     .return_fields("content", "vector_distance")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vector": encode_one(_input)

}
res = client.ft(index_name).search(query, query_params)

table_view(res)

,id,vector_distance,content
0,doc:redispy:81,0.363789558411,"NIKE Brand revenues, which represented over 90..."
1,doc:redispy:74,0.453679800034,"NIKE designs, develops, markets and sells athl..."


## Cleanup
Clean up the index and data.

In [52]:
client.ft(index_name).dropindex(True)

b'OK'